In [1]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.6/191.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 84.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: 

In [1]:
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone

In [2]:
from pyngrok import ngrok
!ngrok authtoken 2lxpqNk7u6Ap5Z9GaqQEj4psoNL_2HNfWPdbK1A6Bg1aP4og9

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
import sys
!{sys.executable} -m pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


In [4]:
import os
from typing import Dict, List
from groq import Groq

os.environ["GROQ_API_KEY"] = "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"

LLAMA3_70B_INSTRUCT = "llama-3.1-70b-versatile"
LLAMA3_8B_INSTRUCT = "llama3.1-8b-instant"

DEFAULT_MODEL = LLAMA3_70B_INSTRUCT

client = Groq()

def shoe_sales_agent(content: str):
    prompt = f"""
    Pretend you are a shoe sales agent, give me a summary of features \
    of the shoe based on input delimited by 3 backticks.
    ```{content}```
    """
    return { "role": "user", "content": prompt }

def chat_completion(
    messages: List[Dict],
    model = DEFAULT_MODEL,
    temperature: float = 0.6,
    top_p: float = 0.9,
) -> str:
    response = client.chat.completions.create(
        messages=messages,
        model=model,
        temperature=temperature,
        top_p=top_p,
    )
    return response.choices[0].message.content

def completion(
    prompt: str,
    model: str = DEFAULT_MODEL,
    temperature: float = 0.6,
    top_p: float = 0.9,
) -> str:
    return chat_completion(
        [shoe_sales_agent(prompt)],
        model=model,
        temperature=temperature,
        top_p=top_p,
    )

def complete_and_print(prompt: str, model: str = DEFAULT_MODEL):
    response = completion(prompt, model)
    return response

In [5]:
pinecone = Pinecone(api_key='d130d359-d4c6-4ce9-b5c8-4e25368899fe')
INDEX_NAME = 'shoe'
index = pinecone.Index(INDEX_NAME)

def pick_shoe_items(shoe):
  result = {}
  result['id']    = shoe.get('id')
  result['name']  = shoe.get('metadata').get('name')
  result['image'] = shoe.get('metadata').get('image')
  result['price'] = shoe.get('metadata').get('price') # shoe['price'] is same as shoe.get('price')
  result['message'] = complete_and_print ( result['name'])
  return result

def run_query(query):
  results = []
  embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
  embedding = embedding_model.encode(query).tolist()
  query_response = index.query(top_k=4, vector=embedding, include_metadata=True, include_values=True)
  results = [pick_shoe_items(match) for match in query_response['matches'] if match]
  results = [r for r in results if r is not None]  # Remove any None results
  return results

In [6]:
pip install flask-cors

In [7]:
from flask import Flask, jsonify, request
from flask_cors import CORS
app = Flask(__name__)
CORS(app)
@app.route('/api/shoes', methods=['GET', 'POST'])
def home():
    if request.method == 'POST':
        query = request.form.get('q') or request.args.get('q')
    else:
        query = request.args.get('q')
    if query is None:
        return jsonify({"error": "No query provided"}), 400
    results = run_query(query)
    if not results:
        return jsonify({"error": "No results found"}), 404
    return jsonify(results)
# Open an ngrok tunnel to the port where your Flask app is running
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")
# Run Flask app (this will block execution, so ensure ngrok is set up first)
app.run()

Public URL: NgrokTunnel: "https://6767-34-48-118-184.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, Fut

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [28/Sep/2024 13:01:33] "POST /api/shoes?q=looking%20for%20baby%20shoes HTTP/1.1" 200 -
